# MCP STDIO Transport

This example demonstrates the raw MCP protocol over STDIO transport. We spawn an MCP server as a subprocess and communicate with it using JSON-RPC messages.

In [ ]:
import subprocess
import json

## Start the MCP Server

The server runs as a subprocess. We communicate via stdin/stdout pipes using newline-delimited JSON-RPC messages.

In [ ]:
proc = subprocess.Popen(
    ['fastmcp', 'run', '-t', 'stdio', 'example_mcp_server.py'],
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True,
    bufsize=1
)

In [ ]:
def send_message(message: dict) -> dict | None:
    """Send a JSON-RPC message and return the response."""
    line = json.dumps(message)
    proc.stdin.write(line + '\n')
    proc.stdin.flush()
    
    # Read response (only for requests with id, not notifications)
    if 'id' in message:
        response = proc.stdout.readline()
        return json.loads(response)

## 1. Initialize

The client sends an `initialize` request with protocol version and capabilities. The server responds with its capabilities.

In [ ]:
init_request = {
    "jsonrpc": "2.0",
    "id": 1,
    "method": "initialize",
    "params": {
        "protocolVersion": "2025-03-26",
        "capabilities": {
            "roots": {"listChanged": True},
            "sampling": {}
        },
        "clientInfo": {
            "name": "ExampleClient",
            "version": "1.0.0"
        }
    }
}

response = send_message(init_request)
print(json.dumps(response, indent=2))

## 2. Initialized Notification

After successful initialization, the client sends a notification to confirm the session is ready. Notifications have no `id` and receive no response.

In [ ]:
initialized_notification = {
    "jsonrpc": "2.0",
    "method": "notifications/initialized"
}

send_message(initialized_notification)
print("Initialized notification sent")

## 3. Ping

A simple health check to verify the server is responsive.

In [ ]:
ping_request = {
    "jsonrpc": "2.0",
    "id": 2,
    "method": "ping"
}

response = send_message(ping_request)
print(json.dumps(response, indent=2))

## 4. List Tools

Discover what tools the server exposes. The response includes tool names, descriptions, and input schemas.

In [ ]:
list_tools_request = {
    "jsonrpc": "2.0",
    "id": 3,
    "method": "tools/list"
}

response = send_message(list_tools_request)
print(json.dumps(response, indent=2))

## 5. Call Tool

Invoke a tool by name with arguments. The server executes the tool and returns the result.

In [ ]:
call_tool_request = {
    "jsonrpc": "2.0",
    "id": 4,
    "method": "tools/call",
    "params": {
        "name": "add",
        "arguments": {
            "a": 40,
            "b": 2
        }
    }
}

response = send_message(call_tool_request)
print(json.dumps(response, indent=2))

## 6. Call Tool with Invalid Arguments

The protocol handles errors gracefully. Here we call `add` with an invalid argument name.

In [ ]:
call_tool_error = {
    "jsonrpc": "2.0",
    "id": 5,
    "method": "tools/call",
    "params": {
        "name": "add",
        "arguments": {
            "a": 40,
            "z": "2"  # Invalid argument
        }
    }
}

response = send_message(call_tool_error)
print(json.dumps(response, indent=2))

## Cleanup

In [ ]:
proc.terminate()
proc.wait()